# <center> Public Data

In [1]:
from platform import python_version

print(python_version())

3.6.8


In [2]:
# ! pip3 install --upgrade pip
# ! pip3 install censusgeocode
# ! pip3 install msgpack
# ! pip3 install pysal
# ! pip3 install cenpy
# ! pip3 install CensusData
# ! pip3 install census
# ! pip3 install us
# ! pip3 install pyusps
# # ! pip3 install usps
# ! pip3 install bs4
# ! pip3 install impyla
# ! pip3 install thrift_sasl==0.2.1
# ! pip3 install sasl
# ! pip3 install sqlalchemy
# ! pip3 install xlrd

In [3]:
from IPython.display import display
from census import Census
import time
import us
import collections
from us import states
import censusdata as cd
import censusgeocode
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import os
import ast
import requests
import datetime
import glob
import zipfile
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from smartystreets_python_sdk import StaticCredentials, exceptions, ClientBuilder
from smartystreets_python_sdk.us_street import Lookup
from pyusps import address_information
# for connecting to Impala
from impala.dbapi import connect
from impala.util import as_pandas
import csv

## <h1><center>Useful functions</center></h1>  

In [4]:
def scale_numeric(fit_df, trans_df, cols):
    df_scaled = trans_df[cols].copy()
    scaler = StandardScaler().fit(fit_df[cols])
    df_scaled = pd.DataFrame(scaler.transform(df_scaled), index=df_scaled.index, columns=cols)
    return(df_scaled)

def scale_encode(fit_df, trans_df):
    num_cols = list(fit_df.columns[(fit_df.dtypes == 'float64') | (fit_df.dtypes == 'int64')])
    str_cols = list(set(fit_df.columns)^set(num_cols))

    if (len(num_cols)>0) & (len(str_cols)>0):
        df_scaled = scale_numeric(fit_df=fit_df, trans_df=trans_df, cols=num_cols)
        df_dummy = pd.get_dummies(trans_df[str_cols])
        df =  df_scaled.join(df_dummy)
    elif len(num_cols)==0:
        df = pd.get_dummies(trans_df[str_cols])
    elif len(str_cols)==0:
        df = scale_numeric(fit_df=fit_df, trans_df=trans_df, cols=num_cols)        
    return(df)

In [5]:
def set_outliers_to_null(df):
    
    num_cols = list(df.columns[(df.dtypes == 'float64') | (df.dtypes == 'int64')])
    str_cols = list(set(df.columns)^set(num_cols))

    for col in list(df.columns):
        if col in num_cols:
            cond1 = (df[col]<0)
            cond2 = (df[col].isnull())
#             cond3 = (df[col]>=(np.mean(df[col])+3*np.std(df[col]))) 
#             cond4 = (df[col]<=(np.mean(df[col])-3*np.std(df[col])))
            df[col][cond1 | cond2] = None

        elif col in str_cols:
            cond1 = (df[col].isnull())
            cond2 = (df[col]=='None')
            df[col][cond1 | cond2] = None
            
    return(df)

def impute_mean(df, na_cols):
    num_cols = list(df.columns[(df.dtypes == 'float64') | (df.dtypes == 'int64')])
    str_cols = list(set(df.columns)^set(num_cols))
    for i in na_cols:
        if i in num_cols:
            df.loc[df[i].isnull(), i] = df[i].dropna().mean()
        elif i in str_cols:
            df.loc[df[i].isnull(), i] = df[i].dropna().mode()            
    return(df)


def handle_nas(df_impute, method='impute_mean', threshold=0.02):
    
    df = set_outliers_to_null(df_impute)

    df = df.drop(columns=list(df.isnull().sum()[np.sum(df.isnull())>0.7*df.shape[0]].index))
    
    # Impute mean for columns with very low % of NAs
    na_rows_in_each_col = np.sum(df.isnull())
    prop_na_each_col = na_rows_in_each_col/df.shape[0]
    thresh_cols = list(prop_na_each_col[prop_na_each_col <= threshold].keys())
    if len(thresh_cols)>0:
        df = impute_mean(df=df, na_cols=thresh_cols)
    
    na_cols = list(df.isnull().sum().sort_values(ascending=True)[np.sum(df.isnull())>0].index)
    num_cols = list(df.columns[(df.dtypes == 'float64') | (df.dtypes == 'int64')])
    str_cols = list(set(df.columns)^set(num_cols))
    predictors = list(set(na_cols)^set(df.columns))
    
    print('Imputing data for variables: %s \n Total number of features for imputing: %s \n Total Columns: %s'
          %(na_cols, len(na_cols), len(list(df.columns))))
    
    # if all cols have na's, impute one with least na's by mean
    if len(predictors) == 0:
        df = impute_mean(df=df, na_cols=na_cols[:1])
        na_cols = list(df.isnull().sum().sort_values(ascending=True)[np.sum(df.isnull())>0].index)
        predictors = list(set(na_cols)^set(df.columns))

    if method == 'impute_mean':
        df = impute_mean(df=df, na_cols=na_cols)
    elif method == 'impute_predict':  
        
        for i in na_cols:
            t0 = time.time()
            print('loop started for %s...'%i)
            X = df[predictors]
            print('using as predictors: %s'%predictors)
            X_scaled = scale_encode(X, X)
            y = df.loc[~df[i].isnull(), i]
            X_train = X_scaled.loc[~df[i].isnull()]
            X_pred = X_scaled.loc[df[i].isnull()]
            print('   scaling done...')
            if i in num_cols:
                model = KNeighborsRegressor().fit(X_train, y)
                print('   model fit...')
                df.loc[df[i].isnull(), i] = model.predict(X_pred)
            elif i in str_cols:
                model = KNeighborsClassifier().fit(X_train, y)
                print('   model fit...')
                df.loc[df[i].isnull(), i] = model.predict(X_pred)
            predictors.append(i)
            print('   %s done...'%i)
            t1 = time.time()
            T = t1-t0
            print('   Time taken: %s'%T)            
            
    return(df)

In [6]:
def get_df_from_impala(query, write_to_path):
    
    conn = connect(host='cloudera-service.us.eap.aon.com', port=21050, 
                   database='discovery_arhs_mscrm', use_ssl=True, auth_mechanism='GSSAPI')
    cursor = conn.cursor()
    cursor.execute(query)

    # write to csv
    columns = [datum[0] for datum in cursor.description]

    with open(write_to_path, 'w', newline='') as outcsv:
        writer = csv.writer(outcsv, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL, lineterminator='\n')
        writer.writerow(columns)
        for row in cursor:
            writer.writerow(row)

    # import as dataframe
    cursor.execute(query)
    df = as_pandas(cursor)
    
    return(df)

In [7]:
def read_all_csv(path):
    fnames = glob.glob(path)
    fskip = []
    df = pd.read_csv(fnames[0], sep = ',', encoding='utf-8')
    
    for f in fnames[1:]:
        try:
            df = pd.concat([df, pd.read_csv(f, sep = ',', encoding='utf-8')])
        except:
            fskip.append(f)
            pass
    
    print("Skipped files list:")
    print(fskip)
    print("%s files read"%(len(fnames)-len(fskip)))
    
    return(df)

## <h1><center>ACS Data (one-time pull)</center></h1> 

### Note: To download 2018 and beyond ACS data:
Census will be changing the website from American FactFinder to https://www.census.gov/data/developers/data-sets/acs-5year.html.
So, a different python package will be required to download the newer data sets.

Links to the new package: 
- https://jtleider.github.io/censusdata/index.html (Homepage)
- https://jtleider.github.io/censusdata/example1.html (Example to download block group level data)

Also see below code.

In [8]:
# to see all geographies
geo = cd.censusgeo([('state', '17'), ('county', '031'), ('block group', '*')])
cd.geographies(within=geo, src='acs5', year=2018)

var = ['B01001_001E', 'B01002_001E', 'B19013_001E', "B01001_002E"]

# to download a table from census website
df_census = cd.download(src='acs5', year=2018, geo=geo, var=var)
df_census

,B01001_001E,B01002_001E,B19013_001E,B01001_002E
"Block Group 1, Census Tract 8088, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:808800> block group:1",1738,39.2,96111,826
"Block Group 2, Census Tract 8088, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:808800> block group:2",1994,47.3,173646,962
"Block Group 2, Census Tract 8092, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:809200> block group:2",1210,34.7,51136,606
"Block Group 4, Census Tract 8092, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:809200> block group:4",945,37.4,-666666666,430
"Block Group 1, Census Tract 8092, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:809200> block group:1",847,37.7,48019,430
...,...,...,...,...
"Block Group 4, Census Tract 2909, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:290900> block group:4",1393,26.0,42024,644
"Block Group 4, Census Tract 8288.01, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:828801> block group:4",926,36.3,75357,435
"Block Group 3, Census Tract 5103, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:510300> block group:3",679,36.5,-666666666,350
"Block Group 2, Census Tract 6304, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:630400> block group:2",1591,27.2,33684,835


**Get All Counties**

In [9]:
# get state county fips in consumable format from the url
url = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.select_one('tbody').select('tr')[1:]
state_county_dict = [    
                        dict(
                            county_fips=t.select('td')[0].text[-3:],
                            state_fips=t.select('td')[0].text[-5:-3])
                            for t in table
                    ]
state_county_df = pd.DataFrame(state_county_dict)
display(state_county_df.shape)
display(state_county_df.head(10))

(3232, 2)

,county_fips,state_fips
0,001,01
1,003,01
2,005,01
3,007,01
4,009,01
5,011,01
6,013,01
7,015,01
8,017,01
9,019,01


In [12]:
# # Search keywoards (eg. occupation)
# cd.search('acs5', 2017, 'label', 'age')

# find tables on census website and determine variables to pull in
cd.censustable('acs1', 2018, 'B27001')
#                , 'B25072', 'B10058')

OrderedDict([('B27001_001E',
              {'concept': 'HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE',
               'label': 'Estimate!!Total',
               'predicateType': 'int'}),
             ('B27001_002E',
              {'concept': 'HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE',
               'label': 'Estimate!!Total!!Male',
               'predicateType': 'int'}),
             ('B27001_003E',
              {'concept': 'HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE',
               'label': 'Estimate!!Total!!Male!!Under 6 years',
               'predicateType': 'int'}),
             ('B27001_004E',
              {'concept': 'HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE',
               'label': 'Estimate!!Total!!Male!!Under 6 years!!With health insurance coverage',
               'predicateType': 'int'}),
             ('B27001_005E',
              {'concept': 'HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE',
               'label': 'Estimate!!Total!!Male!!Under 6 yea

**List Variables for Different Levels**

In [11]:
c = Census("5c7ae754b8fdea4870fe540ef427a762938fdbe2", year=2017)

f_block = (
    'B01001_001E', 'B01001_002E', 'B01001_019E', 'B01001_020E', 
    'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 
    'B01001_025E', 'B01001_026E', 'B01001_043E', 'B01001_044E', 
    'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 
    'B01001_049E', # population above 62
    'B28005_014E', 'B28005_016E', 'B28005_017E', 'B28005_018E',
    'B28005_019E', # presence of internet above 65
    'B09020_001E', 'B09020_002E', 'B09020_003E', 'B09020_004E',
    'B09020_005E', 'B09020_006E', 'B09020_007E', 'B09020_008E',
    'B09020_009E', 'B09020_010E', 'B09020_011E', 'B09020_012E',
    'B09020_013E', 'B09020_014E', 'B09020_015E', 'B09020_016E',
    'B09020_017E', 'B09020_018E', 'B09020_019E', 'B09020_020E',
    'B09020_021E', # relationship by household type above 65
    'B09021_022E', 'B09021_023E', 'B09021_024E', 'B09021_025E', 
    'B09021_026E', 'B09021_027E', 'B09021_028E', # living arrangements above 65
        
    'B19013_001E', # median income
    'B19069_001E', # aggregate retirement income for households
    'B19049_005E', 'B19049_004E', # median income by age 
    'B25077_001E', # median value of house
    'B25039_001E', # median year householder moved into unit
    'B25010_001E', # average household size
    'B27010_051E', 'B27010_052E', 'B27010_053E', 'B27010_054E', 
    'B27010_055E', 'B27010_056E', 'B27010_057E', 'B27010_058E', 
    'B27010_059E', 'B27010_060E', 'B27010_061E', 'B27010_062E',
    'B27010_063E', 'B27010_064E', 'B27010_065E', 'B27010_066E', # Health insurance coverage 
    'B15003_001E', 'B15003_017E', 'B15003_018E', 'B15003_019E', 
    'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E', 
    'B15003_024E', 'B15003_025E', # Education
    'B25003_001E', 'B25003_002E', 'B25003_003E', # Owner/Renter Occupied
    'B25091_002E', 'B25091_013E', # housing units with/without mortgage
    'B25071_001E', # median rent % of income
    'B25092_001E', # median owner costs % of income
    'B25093_023E', 'B25093_024E', 'B25093_025E', 'B25093_026E', 'B25093_027E', 
    'B25093_028E', 'B25093_029E', # owner cost % of income, by age
    'B25072_023E', 'B25072_024E', 'B25072_025E', 'B25072_026E', 'B25072_027E',
    'B25072_028E', 'B25072_029E', # rent % of income, by age
    )

f_tract = (
    'B07001_001E', 'B07001_062E', 'B07001_063E', 'B07001_064E',
    'B07001_078E', 'B07001_079E', 'B07001_080E', # moved state/county in past year by age
    'B25105_001E', # median monthly housing cost
    'B10058_001E', 'B10058_005E', 'B10058_010E', # Grand parents taking care of grand children >60 yrs
    'B08126_001E', 'B08126_002E','B08126_003E', 'B08126_004E','B08126_005E', 'B08126_006E',
    'B08126_007E', 'B08126_008E','B08126_009E', 'B08126_010E','B08126_011E', 'B08126_012E',
    'B08126_013E', 'B08126_014E','B08126_015E', # industry 
    'B08124_002E', 'B08124_003E', 'B08124_004E', 'B08124_005E', 'B08124_006E', 'B08124_007E', # occupation
    'B08124_001E', 'B08124_008E', 'B08124_015E', 'B08124_022E', 'B08124_029E',
    'B08124_036E', 'B08124_043E', # means of transport
    
    'B27001_021E', 'B27001_022E', 'B27001_023E', # male, 55-64: total/with/without health insurance
    'B27001_024E', 'B27001_025E', 'B27001_026E', # male, 65-74: total/with/without health insurance
    'B27001_027E', 'B27001_028E', 'B27001_029E', # male, 75+: total/with/without health insurance
    'B27001_049E', 'B27001_050E', 'B27001_051E', # female, 55-64: total/with/without health insurance
    'B27001_052E', 'B27001_053E', 'B27001_054E', # female, 65-74: total/with/without health insurance
    'B27001_055E', 'B27001_056E', 'B27001_057E', # female, 75+: total/with/without health insurance
    
    )

common_fields = list(f_block)

acs_fields = list(np.unique(f_block+f_tract))

counter = 0

In [12]:
# dict_list_tract_sample = c.acs5.state_county_tract(fields=acs_fields, state_fips=state_county_df['state_fips'][7], 
#                                                    county_fips=state_county_df['county_fips'][7], tract='*')
# df_tract_sample = pd.DataFrame(dict_list_tract_sample)
# display(np.sum(df_tract_sample.isnull())[np.sum(df_tract_sample.isnull())>0])
# display(df_tract_sample.shape)
# display(np.sum(df_tract_sample<0)[np.sum(df_tract_sample<0)>0])

In [13]:
# dict_list_subdiv_sample = c.acs5.state_county_subdivision(fields=acs_fields, state_fips=state_county_df['state_fips'][7], 
#                                                          county_fips=state_county_df['county_fips'][7], subdiv_fips='*')
# df_subdiv_sample = pd.DataFrame(dict_list_subdiv_sample)
# display(np.sum(df_subdiv_sample.isnull())[np.sum(df_subdiv_sample.isnull())>0])
# display(df_subdiv_sample)
# display(np.sum(df_subdiv_sample<0)[np.sum(df_subdiv_sample<0)>0])

In [14]:
# dict_list_block_sample = c.acs5.state_county_blockgroup(fields=f_block, state_fips=state_county_df['state_fips'][7], 
#                                                         county_fips=state_county_df['county_fips'][7], blockgroup='*')
# df_block_sample = pd.DataFrame(dict_list_block_sample)
# display(np.sum(df_block_sample.isnull())[np.sum(df_block_sample.isnull())>0])
# display(df_block_sample.shape)
# display(np.sum(df_block_sample<0)[np.sum(df_block_sample<0)>0])

**Make API Connections and Pull Specific Variables**

In [15]:
for i in range(state_county_df.shape[0]):
    
    state = state_county_df['state_fips'][i]
    county = state_county_df['county_fips'][i]
    skip_county_block = []
    skip_county_tract = []
    
    # blockgroup
    dict_list_block = c.acs5.state_county_blockgroup(fields=f_block, state_fips=state, 
                                                     county_fips=county, blockgroup='*')
    if len(dict_list_block)!=0:
        df_block = pd.DataFrame(dict_list_block)
        df_block.to_csv('output/census/block_level_%s.csv'%i, sep = ',', 
                        encoding='utf-8', index=False)
    else:        
        skip_county_block.append(i)
    
    # tract
    dict_list_tract = c.acs5.state_county_tract(fields=acs_fields, state_fips=state, 
                                                county_fips=county, tract='*')
    if len(dict_list_tract)!=0:
        df_tract = pd.DataFrame(dict_list_tract)
        df_tract.to_csv('output/census/tract_level_%s.csv'%i, sep = ',', 
                        encoding='utf-8', index=False)
    else:
        skip_county_tract.append(i)
    
    counter += 1
    if counter%20:
        time.sleep(5)
    
    print(i)
    
print("Skipped blocks in county:")
print(skip_county_block)
print("Skipped tracts in county:")
print(skip_county_tract)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


**Write Blockgroup File**

In [16]:
# Read and stack all CSVs in path, printing filenames that resulted in error
path = 'output/census/block_level_*.csv'
df_block = read_all_csv(path)

Skipped files list:
[]
3209 files read


In [17]:
# Remove unwanted columns and write to csv
exc_cols = ["Unnamed: 0.1", "Unnamed: 0"]
inc_cols = list(set(df_block.columns) - set(exc_cols))
df_block = set_outliers_to_null(df_block)
df_block[inc_cols].to_csv('output/census_data_blockgroup2.csv', sep = ',', 
                          encoding='utf-8', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [18]:
df_block = pd.read_csv('output/census_data_blockgroup2.csv', sep = ',', encoding='utf-8')
display(df_block.head(10))
display(df_block.describe())
display(df_block.shape)
display(df_block.columns)
display(np.sum(df_block.isnull())[np.sum(df_block.isnull())>0])
# display(np.sum(df_block==0))
display(np.sum(df_block<0)[np.sum(df_block<0)>0])

,B28005_018E,B27010_059E,B25093_028E,B01001_001E,B25093_024E,B27010_066E,B27010_064E,B27010_055E,B25072_025E,B01001_023E,...,B27010_062E,B09020_020E,B25003_002E,B01001_043E,B01001_049E,B25003_003E,B01001_022E,B15003_023E,B15003_018E,B28005_014E
0,26.0,0.0,52.0,1222,101.0,0.0,28.0,83.0,0.0,50.0,...,0.0,9.0,474.0,82.0,33.0,69.0,48.0,67.0,19.0,461.0
1,0.0,6.0,11.0,2244,34.0,0.0,0.0,23.0,6.0,0.0,...,5.0,0.0,360.0,40.0,28.0,243.0,29.0,0.0,52.0,95.0
2,12.0,0.0,10.0,2059,8.0,0.0,0.0,38.0,0.0,31.0,...,57.0,9.0,282.0,17.0,21.0,198.0,14.0,13.0,9.0,122.0
3,8.0,0.0,0.0,1843,23.0,0.0,0.0,8.0,0.0,39.0,...,80.0,0.0,381.0,69.0,0.0,92.0,34.0,24.0,78.0,138.0
4,17.0,0.0,15.0,2450,24.0,0.0,0.0,57.0,0.0,10.0,...,0.0,0.0,298.0,24.0,26.0,365.0,24.0,19.0,24.0,94.0
5,10.0,0.0,32.0,2553,39.0,0.0,0.0,131.0,0.0,0.0,...,12.0,0.0,669.0,34.0,15.0,107.0,58.0,53.0,15.0,321.0
6,0.0,0.0,9.0,2322,0.0,15.0,0.0,26.0,0.0,0.0,...,0.0,15.0,352.0,17.0,0.0,193.0,0.0,102.0,26.0,57.0
7,0.0,0.0,18.0,1164,33.0,0.0,10.0,8.0,0.0,9.0,...,0.0,0.0,294.0,34.0,0.0,43.0,15.0,16.0,19.0,106.0
8,18.0,0.0,0.0,645,0.0,0.0,0.0,78.0,0.0,0.0,...,18.0,32.0,75.0,29.0,64.0,165.0,14.0,47.0,0.0,124.0
9,16.0,0.0,0.0,1501,83.0,0.0,0.0,71.0,0.0,0.0,...,0.0,0.0,338.0,69.0,16.0,178.0,53.0,57.0,31.0,168.0


,B28005_018E,B27010_059E,B25093_028E,B01001_001E,B25093_024E,B27010_066E,B27010_064E,B27010_055E,B25072_025E,B01001_023E,...,B27010_062E,B09020_020E,B25003_002E,B01001_043E,B01001_049E,B25003_003E,B01001_022E,B15003_023E,B15003_018E,B28005_014E
count,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,...,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000,220200.000000
mean,16.503933,0.186076,21.987725,1472.748015,58.363933,1.851344,5.054033,57.879382,2.398910,16.823274,...,18.903215,3.559655,347.997557,26.670145,18.516916,196.905522,24.540409,82.929696,39.309886,212.587838
std,24.513855,1.919480,27.345924,959.370187,60.552420,8.403779,10.646207,60.654175,7.570364,22.644878,...,33.702939,10.779853,266.055896,27.007727,28.980581,204.244476,29.106770,104.707096,50.913702,179.488683
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,891.000000,21.000000,0.000000,0.000000,20.000000,0.000000,0.000000,...,0.000000,0.000000,183.000000,8.000000,0.000000,64.000000,6.000000,18.000000,10.000000,109.000000
50%,9.000000,0.000000,15.000000,1265.000000,46.000000,0.000000,0.000000,44.000000,0.000000,11.000000,...,8.000000,0.000000,298.000000,20.000000,10.000000,135.000000,18.000000,50.000000,27.000000,178.000000
75%,24.000000,0.000000,31.000000,1808.000000,81.000000,0.000000,7.000000,78.000000,0.000000,24.000000,...,24.000000,0.000000,452.000000,38.000000,24.000000,259.000000,35.000000,112.000000,54.000000,273.000000
max,1780.000000,121.000000,2933.000000,51872.000000,10598.000000,360.000000,759.000000,8537.000000,198.000000,3287.000000,...,1068.000000,844.000000,20473.000000,2105.000000,1275.000000,7146.000000,4708.000000,7209.000000,2705.000000,30284.000000


(220200, 108)

Index(['B28005_018E', 'B27010_059E', 'B25093_028E', 'B01001_001E',
       'B25093_024E', 'B27010_066E', 'B27010_064E', 'B27010_055E',
       'B25072_025E', 'B01001_023E',
       ...
       'B27010_062E', 'B09020_020E', 'B25003_002E', 'B01001_043E',
       'B01001_049E', 'B25003_003E', 'B01001_022E', 'B15003_023E',
       'B15003_018E', 'B28005_014E'],
      dtype='object', length=108)

B25077_001E    13908
B25039_001E     2887
B19069_001E    28584
B19013_001E     7292
B25071_001E    35415
B19049_004E    34431
B19049_005E    56231
B25092_001E     8768
B25010_001E     1642
dtype: int64

Series([], dtype: int64)

In [19]:
# ## Test check from ACS website
# state = (df_block["state"]==17) # illinois
# county = (df_block["county"]==97) # lake county
# tract = (df_block["tract"]==860103)
# blockgroup = (df_block["block group"]==3)
# df_block[["B25071_001E", "block group", "county", "tract"]][state&county&tract&blockgroup]

In [20]:
null = list(np.sum(df_block.isnull())[np.sum(df_block.isnull())>0].keys())
not_null = set(df_block.columns)-set(null)
set(not_null) - set(f_block)

{'block group', 'county', 'state', 'tract'}

**Write Tract File**

In [21]:
# Read and stack all CSVs in path, printing filenames that resulted in error
path = 'output/census/tract_level_*.csv'
df_tract = read_all_csv(path)

Skipped files list:
[]
3209 files read


In [22]:
# Remove unwanted columns and write to csv
exc_cols = ["Unnamed: 0.1", "Unnamed: 0"]
inc_cols = list(set(df_tract.columns) - set(exc_cols))
df_tract = set_outliers_to_null(df_tract)
df_tract[inc_cols].to_csv('output/census_data_tract2.csv', sep = ',', 
                encoding='utf-8', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [23]:
df_tract = pd.read_csv('output/census_data_tract2.csv', sep = ',', encoding='utf-8')
display(df_tract.head(10))
display(df_tract.shape)
display(df_tract.columns)
display(np.sum(df_tract.isnull())[np.sum(df_tract.isnull())>0])
# display(np.sum(df_tract==0))
display(np.sum(df_tract<0)[np.sum(df_tract<0)>0])

,B28005_018E,B27010_059E,B25093_028E,B08124_008E,B27010_064E,B27001_051E,B08124_007E,B25105_001E,B27010_055E,B28005_016E,...,B25093_027E,B09020_019E,B09020_001E,B25093_029E,B25091_002E,B09020_020E,B25003_002E,B09021_022E,B08124_043E,B28005_014E
0,59.0,0.0,76.0,1330.0,9.0,73.0,0.0,595.0,143.0,0.0,...,0.0,0.0,551.0,0.0,374.0,13.0,829.0,478.0,6.0,478.0
1,281.0,0.0,30.0,2735.0,40.0,166.0,0.0,508.0,367.0,24.0,...,13.0,2.0,1312.0,0.0,1008.0,42.0,2321.0,1312.0,183.0,1312.0
2,114.0,0.0,90.0,2902.0,16.0,54.0,10.0,844.0,187.0,18.0,...,10.0,5.0,936.0,2.0,1353.0,14.0,2238.0,936.0,161.0,936.0
3,44.0,0.0,39.0,956.0,6.0,21.0,0.0,620.0,138.0,0.0,...,4.0,3.0,457.0,0.0,388.0,2.0,811.0,457.0,65.0,457.0
4,232.0,0.0,25.0,4036.0,34.0,23.0,0.0,1034.0,290.0,47.0,...,36.0,0.0,1418.0,0.0,1907.0,0.0,3062.0,1418.0,165.0,1418.0
5,92.0,0.0,31.0,2477.0,39.0,0.0,0.0,695.0,397.0,32.0,...,0.0,0.0,1269.0,8.0,941.0,13.0,1714.0,1194.0,95.0,1194.0
6,120.0,0.0,15.0,2090.0,56.0,67.0,0.0,663.0,296.0,14.0,...,35.0,13.0,1024.0,0.0,861.0,28.0,1696.0,952.0,59.0,952.0
7,34.0,0.0,83.0,2622.0,13.0,32.0,0.0,1014.0,179.0,11.0,...,9.0,0.0,708.0,0.0,1122.0,0.0,1658.0,676.0,61.0,676.0
8,10.0,0.0,17.0,277.0,5.0,32.0,0.0,460.0,27.0,5.0,...,0.0,0.0,123.0,0.0,32.0,0.0,187.0,123.0,13.0,123.0
9,23.0,0.0,34.0,1437.0,13.0,27.0,0.0,584.0,103.0,0.0,...,29.0,0.0,440.0,11.0,318.0,0.0,754.0,440.0,0.0,440.0


(73951, 164)

Index(['B28005_018E', 'B27010_059E', 'B25093_028E', 'B08124_008E',
       'B27010_064E', 'B27001_051E', 'B08124_007E', 'B25105_001E',
       'B27010_055E', 'B28005_016E',
       ...
       'B25093_027E', 'B09020_019E', 'B09020_001E', 'B25093_029E',
       'B25091_002E', 'B09020_020E', 'B25003_002E', 'B09021_022E',
       'B08124_043E', 'B28005_014E'],
      dtype='object', length=164)

B25105_001E    1015
B25077_001E    2010
B25039_001E    2287
B19069_001E    1711
B19013_001E    1052
B07001_001E     928
B25071_001E    1630
B19049_004E    2040
B07001_079E     928
B25092_001E    1665
B25010_001E     902
B07001_078E     928
B07001_063E     928
B07001_080E     928
B07001_062E     928
B07001_064E     928
B19049_005E    4077
dtype: int64

Series([], dtype: int64)

**Write Zip File**

In [24]:
dict_list_zip = c.acs5.zipcode(fields=acs_fields, zcta="*")
df_zip = pd.DataFrame(dict_list_zip)
df_zip = set_outliers_to_null(df_zip)
df_zip.to_csv('output/census_data_zip2.csv', sep = ',', 
              encoding='utf-8', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [25]:
df_zip = pd.read_csv('output/census_data_zip2.csv', sep = ',', encoding='utf-8')
display(df_zip.head(10))
display(df_zip.shape)
display(df_zip.columns)
display(np.sum(df_zip.isnull())[np.sum(df_zip.isnull())>0])
display(np.sum(df_zip<0)[np.sum(df_zip<0)>0])

,B01001_001E,B01001_002E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_026E,...,B27010_062E,B27010_063E,B27010_064E,B27010_065E,B27010_066E,B28005_014E,B28005_016E,B28005_017E,B28005_018E,B28005_019E
0,345,192.0,0.0,0.0,23.0,0.0,0.0,0.0,10.0,153.0,...,0.0,0.0,0.0,18.0,0.0,69.0,0.0,45.0,0.0,24.0
1,421,188.0,33.0,25.0,13.0,2.0,0.0,5.0,4.0,233.0,...,0.0,0.0,6.0,4.0,0.0,129.0,21.0,100.0,0.0,8.0
2,211,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,116.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,10.0,0.0,0.0
3,203,103.0,0.0,0.0,0.0,0.0,6.0,8.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,69.0,0.0,43.0,0.0,26.0
4,224,154.0,0.0,0.0,0.0,7.0,0.0,0.0,7.0,70.0,...,0.0,0.0,0.0,0.0,0.0,36.0,0.0,22.0,14.0,0.0
5,4510,2277.0,19.0,52.0,52.0,26.0,48.0,32.0,35.0,2233.0,...,36.0,0.0,9.0,81.0,2.0,499.0,18.0,322.0,36.0,123.0
6,1649,799.0,26.0,34.0,19.0,17.0,13.0,0.0,5.0,850.0,...,16.0,0.0,0.0,9.0,0.0,173.0,0.0,76.0,37.0,60.0
7,199,101.0,12.0,4.0,11.0,6.0,0.0,0.0,2.0,98.0,...,0.0,0.0,0.0,4.0,0.0,47.0,0.0,41.0,4.0,2.0
8,364,214.0,20.0,5.0,7.0,5.0,4.0,6.0,9.0,150.0,...,2.0,0.0,3.0,6.0,0.0,69.0,4.0,40.0,7.0,18.0
9,9968,4994.0,181.0,93.0,268.0,139.0,140.0,86.0,72.0,4974.0,...,91.0,0.0,150.0,230.0,0.0,1638.0,0.0,1298.0,116.0,224.0


(33120, 162)

Index(['B01001_001E', 'B01001_002E', 'B01001_019E', 'B01001_020E',
       'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E',
       'B01001_025E', 'B01001_026E',
       ...
       'B27010_062E', 'B27010_063E', 'B27010_064E', 'B27010_065E',
       'B27010_066E', 'B28005_014E', 'B28005_016E', 'B28005_017E',
       'B28005_018E', 'B28005_019E'],
      dtype='object', length=162)

B07001_001E     131
B07001_062E     131
B07001_063E     131
B07001_064E     131
B07001_078E     131
B07001_079E     131
B07001_080E     131
B19013_001E    2164
B19049_004E    3956
B19049_005E    4501
B19069_001E    3527
B25010_001E     728
B25039_001E    1244
B25071_001E    5538
B25077_001E    2542
B25092_001E    1568
B25105_001E    1857
dtype: int64

Series([], dtype: int64)

## <h1><center>Data Transformations</center></h1> 

In [26]:
sample1 = pd.DataFrame({"A": [1,2,3,4], "B": [2,np.NaN,6,np.NaN]})
sample2 = pd.DataFrame({"C": [1,2,3,4], "D": [np.NaN,2,np.NaN,6]})
display(sample1)
display(sample2)
display(sample1["B"].fillna(sample2["D"]))
# NOTE: original DF remains unchanged
display(sample1)

,A,B
0,1,2.0
1,2,NaN
2,3,6.0
3,4,NaN


,C,D
0,1,NaN
1,2,2.0
2,3,NaN
3,4,6.0


0    2.0
1    2.0
2    6.0
3    6.0
Name: B, dtype: float64

,A,B
0,1,2.0
1,2,NaN
2,3,6.0
3,4,NaN


In [27]:
sql_state_county = """

select exch_stateidname as state_name, 
exch_displayname as county_name,
exch_statefipscode as state_fips,
exch_countyfips as county_fips
from
v_exch_county a
left join
v_exch_state b
on a.exch_stateid = b.exch_stateid

"""

In [28]:
# Join state-county FIPS from CRM with CMS data
df_state_county = get_df_from_impala(query=sql_state_county, write_to_path='data/state_county.xlsx')
df_cms_raw = pd.read_excel('data/cms_data_ma_pdp.xlsx')

df_cms = pd.merge(df_state_county, df_cms_raw, how="inner", 
                  left_on=["state_name","county_name"],
                  right_on=["State","County"]).drop(columns=["state_name","county_name","State","County"])

In [29]:
display(df_state_county.shape)
display(df_cms.shape)

(3297, 4)

(3127, 32)

In [30]:
# val_cols = list(set(df_cms.columns)-set(['state_fips', 'county_fips']))
# df_cms['county_fips'].str[-3:]

In [31]:
def get_last3_character(column):
    return(column[0][-3:])

df_cms["county_fips"] = df_cms["county_fips"].str[-3:]
df_cms = df_cms.replace(to_replace=["+","*"], value=None)
display(np.sum(df_cms.isnull())[np.sum(df_cms.isnull())>0].sort_values(ascending=False))
df_cms.head(10)

Series([], dtype: int64)

,state_fips,county_fips,2015 Original Medicare,2015 Medicare Advantage & Other Health Plans,2015 Total Med,2015 Prescription Drug Plans,2015 Medicare Advantage Prescription Drug,2015 Total PDP,2016 Original Medicare,2016 Medicare Advantage & Other Health Plans,...,2018 Total Med,2018 Prescription Drug Plans,2018 Medicare Advantage Prescription Drug,2018 Total PDP,2019 Original Medicare,2019 Medicare Advantage & Other Health Plans,2019 Total Med,2019 Prescription Drug Plans,2019 Medicare Advantage Prescription Drug,2019 Total PDP
0,38,035,7828,1464,9292,5721,479,6200,7896,1671,...,10261,6332,651,6983,8612,1935,10547,6513,709,7222
1,01,077,17796,2799,20595,12179,2535,14714,17776,3196,...,21638,10439,5483,15921,15067,6966,22033,10069,6385,16454
2,01,037,2126,360,2486,1460,342,1802,2164,386,...,2635,1088,886,1973,1825,1226,3051,1123,1188,2312
3,01,101,28477,11254,39731,15619,10970,26589,28759,11901,...,42538,9691,19284,28975,22114,20961,43076,9209,20443,29653
4,38,057,1338,293,1631,1063,40,1103,1312,344,...,1722,1128,44,1172,1463,297,1760,1189,50,1239
5,36,117,9041,11219,20261,5754,10650,16403,9056,11616,...,21518,5924,11860,17784,8650,12963,21612,5557,12490,18047
6,01,131,2767,313,3079,2138,288,2427,2795,331,...,3186,1887,641,2527,2252,854,3106,1703,806,2509
7,01,043,14227,4332,18558,10061,4069,14131,14410,4561,...,19557,8164,6889,15052,11801,7935,19736,7747,7594,15341
8,38,077,2262,630,2892,1918,207,2126,2172,765,...,3112,2071,209,2280,2404,828,3231,2160,237,2397
9,01,045,8810,1039,9850,4487,953,5440,8935,1123,...,10392,3466,2459,5924,7461,3188,10649,3159,3041,6200


In [32]:
df_cms_int = df_cms.astype(int)
display(np.sum(df_cms_int==0)[np.sum(df_cms_int==0)>0].sort_values(ascending=False))

2016 Medicare Advantage Prescription Drug       11
2016 Medicare Advantage & Other Health Plans    10
2019 Medicare Advantage Prescription Drug        8
2018 Medicare Advantage Prescription Drug        8
2015 Medicare Advantage Prescription Drug        7
2018 Medicare Advantage & Other Health Plans     7
2017 Medicare Advantage Prescription Drug        6
2015 Medicare Advantage & Other Health Plans     6
2019 Medicare Advantage & Other Health Plans     4
2017 Medicare Advantage & Other Health Plans     4
2017 Total PDP                                   1
2018 Original Medicare                           1
2017 Prescription Drug Plans                     1
2017 Original Medicare                           1
2016 Total PDP                                   1
2018 Total Med                                   1
2016 Prescription Drug Plans                     1
2016 Total Med                                   1
2018 Prescription Drug Plans                     1
2016 Original Medicare         

In [33]:
df_block = pd.read_csv('output/census_data_blockgroup2.csv', sep = ',', encoding='utf-8')
df_tract = pd.read_csv('output/census_data_tract2.csv', sep = ',', encoding='utf-8')
df_zip = pd.read_csv('output/census_data_zip2.csv', sep = ',', encoding='utf-8')

In [34]:
block_cols = ["state", "county", "tract", "block group"]
tract_cols = ["state", "county", "tract"]
zip_cols = ["zip code tabulation area"]
cms_cols = ["state_fips", "county_fips"]

**Transform county-level fields (CMS Data)**

In [35]:
## Create a county-level above 65 count to use as denominator later.
# get columns for age above 65 from tract table
age_cols = ["B01001_020E", "B01001_021E", "B01001_022E",
            "B01001_023E", "B01001_024E", "B01001_025E",
            "B01001_044E", "B01001_045E", "B01001_046E",
            "B01001_047E", "B01001_048E", "B01001_049E",
            "county"
           ]  

df_tract_gb = df_tract.groupby(by="county", axis="index").sum().reset_index()[age_cols]
        
df_tract_gb

,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,county
0,78388.0,101634.0,126728.0,82748.0,58391.0,53082.0,86000.0,115193.0,149554.0,107341.0,82571.0,97874.0,1
1,105447.0,136831.0,174146.0,117687.0,79611.0,74161.0,117620.0,154252.0,200677.0,144742.0,112205.0,138140.0,3
2,73053.0,94817.0,118439.0,81652.0,52640.0,45397.0,83032.0,106992.0,142013.0,103354.0,76059.0,94291.0,5
3,192.0,321.0,544.0,410.0,216.0,92.0,422.0,366.0,441.0,439.0,194.0,345.0,6
4,38193.0,50673.0,63678.0,43489.0,29217.0,25660.0,41060.0,55600.0,73455.0,52725.0,42575.0,53604.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,1061.0,1044.0,1234.0,848.0,413.0,507.0,770.0,1481.0,1612.0,1198.0,628.0,963.0,800
316,4184.0,5022.0,6060.0,4222.0,2639.0,2428.0,4809.0,6155.0,7693.0,4934.0,4489.0,4704.0,810
317,249.0,271.0,397.0,346.0,184.0,188.0,251.0,380.0,548.0,501.0,432.0,274.0,820
318,117.0,206.0,212.0,181.0,141.0,97.0,160.0,134.0,370.0,202.0,180.0,159.0,830


In [36]:
# join CMS data with above dataframe
df_cms_tract_join = pd.merge(df_cms_int, df_tract_gb, how="left", 
                             left_on=["county_fips"],
                             right_on=["county"])
display(np.sum(df_cms_tract_join==0)[np.sum(df_cms_tract_join==0)>0].sort_values(ascending=False))

2016 Medicare Advantage Prescription Drug       11
2016 Medicare Advantage & Other Health Plans    10
2018 Medicare Advantage Prescription Drug        8
2019 Medicare Advantage Prescription Drug        8
2015 Medicare Advantage Prescription Drug        7
2018 Medicare Advantage & Other Health Plans     7
2017 Medicare Advantage Prescription Drug        6
2015 Medicare Advantage & Other Health Plans     6
2019 Medicare Advantage & Other Health Plans     4
2017 Medicare Advantage & Other Health Plans     4
B01001_024E                                      3
2018 Prescription Drug Plans                     1
2016 Total PDP                                   1
2016 Original Medicare                           1
B01001_020E                                      1
2016 Total Med                                   1
2016 Prescription Drug Plans                     1
2017 Original Medicare                           1
2018 Total Med                                   1
2017 Total Med                 

In [37]:
df = df_cms_tract_join.copy()
out_path = "output/cms_data_county3.csv"
inc_cols = cms_cols

df['percent_medicare_ma'] = df["2019 Medicare Advantage & Other Health Plans"]/df["2019 Total Med"]

df['percent_medicare_ma_above65'] = df["2017 Total Med"]/(df['B01001_020E']+df['B01001_021E']+df['B01001_022E']+
                                     df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
                                     df['B01001_044E']+df['B01001_045E']+df['B01001_046E']+
                                     df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])


cols = list(set(df.columns)-set(df_cms_tract_join.columns))+inc_cols

df[cols].to_csv(out_path, sep = ',', encoding='utf-8', index=False)
display(df[cols].shape)
display(np.sum(df[cols].isnull())[np.sum(df[cols].isnull())>0].sort_values(ascending=False))
display(np.sum(df[cols]<0)[np.sum(df[cols]<0)>0].sort_values(ascending=False))

(3127, 4)

percent_medicare_ma_above65    4
dtype: int64

Series([], dtype: int64)

**Tranform zip-level fields**

In [18]:
cd.censustable('acs5', 2017, 'B25105')

OrderedDict([('B25105_001E',
              {'concept': 'MEDIAN MONTHLY HOUSING COSTS (DOLLARS)',
               'label': 'Estimate!!Median monthly housing costs',
               'predicateType': 'int'})])

In [39]:
df = df_zip.copy()
out_path = "output/census_data_zip3.csv"
inc_cols = ["zip code tabulation area"]

## Blockgroup fields

# % population above 62

df["age_62_64"] = (df['B01001_019E']+df['B01001_043E'])/(df['B01001_002E']+df['B01001_026E'])

df["age_above65"] = (df['B01001_020E']+df['B01001_021E']+df['B01001_022E']+
                     df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
                     df['B01001_044E']+df['B01001_045E']+df['B01001_046E']+
                     df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])/(df['B01001_002E']+df['B01001_026E'])

df["avg_age_above65"] = ((df['B01001_020E']*65.5+df['B01001_021E']*68+df['B01001_022E']*72+
                         df['B01001_023E']*77+df['B01001_024E']*82+df['B01001_025E']*90+
                         df['B01001_044E']*65.5+df['B01001_045E']*68+df['B01001_046E']*72+
                         df['B01001_047E']*77+df['B01001_048E']*82+df['B01001_049E']*90)/
                         (df['B01001_020E']+df['B01001_021E']+df['B01001_022E']+
                         df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
                         df['B01001_044E']+df['B01001_045E']+df['B01001_046E']+
                         df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])
                        )

# presence of computer and type of internet
df["with_internet"] = (df['B28005_016E']+df['B28005_017E'])/df['B28005_014E']

# 65+ living arrangements
df['living_in_group65'] = (df['B09020_021E']>0)
df['living_in_group65'] = df['living_in_group65'].astype(int)
df['living_with_spouse_partner_parent65'] = ((df['B09021_024E']+df['B09021_025E']+
                                              df['B09021_026E'])/df['B09021_022E']+
                                             df['B09020_009E']/df['B09020_001E'])

df['living_with_other65'] = (df['B09021_027E']+df['B09021_028E'])/df['B09021_022E']


# rent/owner costs
df["rent_percent_income65"] = (df["B25072_024E"]*10/df["B25072_023E"] +
                             df["B25072_025E"]*22.5/df["B25072_023E"] +
                             df["B25072_026E"]*27.5/df["B25072_023E"] +
                             df["B25072_027E"]*32.5/df["B25072_023E"] +
                             df["B25072_028E"]*45/df["B25072_023E"])/100
df["owner_cost_percent_income65"] = (df["B25093_024E"]*10/df["B25093_023E"] +
                                     df["B25093_025E"]*22.5/df["B25093_023E"] +
                                     df["B25093_026E"]*27.5/df["B25093_023E"] +
                                     df["B25093_027E"]*32.5/df["B25093_023E"] +
                                     df["B25093_028E"]*45/df["B25093_023E"])/100
df["rent_percent_income"] = df["B25071_001E"]
df["owner_cost_percent_income"] = df["B25092_001E"]

# education
df["education_score"] = ((df["B15003_017E"]+df["B15003_018E"])*2+
                         (df["B15003_019E"]+df["B15003_020E"])*3+
                         df["B15003_021E"]*4+
                         df["B15003_022E"]*5+
                         (df["B15003_023E"]+df["B15003_024E"]+df["B15003_025E"])*6+
                         df["B15003_001E"]-(df["B15003_017E"]+df["B15003_018E"]+df["B15003_019E"]+
                                            df["B15003_020E"]+df["B15003_021E"]+df["B15003_022E"]+
                                            df["B15003_023E"]+df["B15003_024E"]+df["B15003_025E"])
                        )/df["B15003_001E"]

# income/household
df["median_income_above65"] = df["B19049_005E"]
df["median_income"] = df["B19013_001E"]
df["same_address_yrs"] = datetime.datetime.now().year-df["B25039_001E"]

# housing
df['renter_occupied'] = df['B25003_003E']/df['B25003_001E']
df["household_size"] = df["B25010_001E"]
df["house_value"] = df["B25077_001E"]

# insurance

df["insurance_employer_direct"] = (df["B27010_053E"]+df["B27010_054E"]+df["B27010_059E"]+
                                   df["B27010_060E"]+df["B27010_061E"])/df["B27010_051E"]
df["insurance_military_va"] = (df["B27010_056E"]+df["B27010_057E"])/df["B27010_051E"]
df["insurance_none"] = df["B27010_066E"]/df["B27010_051E"]


## Tract level fields

# monthly housing costs
df['monthly_housing_cost'] = df['B25105_001E']

# moved state/county
df['moved_state_county65'] = (df['B07001_062E']+df['B07001_063E']+df['B07001_064E']+
                             df['B07001_078E']+df['B07001_079E']+df['B07001_080E'])/df['B07001_001E']

cols = list(set(df.columns)-set(df_zip.columns))+inc_cols
df[cols].to_csv(out_path, sep = ',', 
              encoding='utf-8', index=False)
display(df[cols].shape)
display(np.sum(df[cols].isnull())[np.sum(df[cols].isnull())>0].sort_values(ascending=False))
display(np.sum(df[cols]<0)[np.sum(df[cols]<0)>0].sort_values(ascending=False))

(33120, 24)

rent_percent_income65                  5922
rent_percent_income                    5538
median_income_above65                  4501
house_value                            2542
median_income                          2164
monthly_housing_cost                   1857
owner_cost_percent_income              1568
owner_cost_percent_income65            1422
same_address_yrs                       1244
living_with_other65                    1142
living_with_spouse_partner_parent65    1142
with_internet                          1142
insurance_military_va                  1094
avg_age_above65                        1046
insurance_none                          779
household_size                          728
renter_occupied                         573
moved_state_county65                    446
education_score                         358
age_above65                             317
age_62_64                               317
insurance_employer_direct               112
dtype: int64

Series([], dtype: int64)

**Transform tract-level fields**

In [40]:
df = df_tract.copy()
out_path = "output/census_data_tract3.csv"
inc_cols = tract_cols

## Blockgroup fields

# % population above 62

df["age_62_64"] = (df['B01001_019E']+df['B01001_043E'])/(df['B01001_002E']+df['B01001_026E'])

df["age_above65"] = (df['B01001_020E']+df['B01001_021E']+df['B01001_022E']+
                     df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
                     df['B01001_044E']+df['B01001_045E']+df['B01001_046E']+
                     df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])/(df['B01001_002E']+df['B01001_026E'])

df["avg_age_above65"] = ((df['B01001_020E']*65.5+df['B01001_021E']*68+df['B01001_022E']*72+
                         df['B01001_023E']*77+df['B01001_024E']*82+df['B01001_025E']*90+
                         df['B01001_044E']*65.5+df['B01001_045E']*68+df['B01001_046E']*72+
                         df['B01001_047E']*77+df['B01001_048E']*82+df['B01001_049E']*90)/
                         (df['B01001_020E']+df['B01001_021E']+df['B01001_022E']+
                         df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
                         df['B01001_044E']+df['B01001_045E']+df['B01001_046E']+
                         df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])
                        )

# presence of computer and type of internet
df["with_internet"] = (df['B28005_016E']+df['B28005_017E'])/df['B28005_014E']

# 65+ living arrangements
df['living_in_group65'] = (df['B09020_021E']>0)
df['living_in_group65'] = df['living_in_group65'].astype(int)
df['living_with_spouse_partner_parent65'] = ((df['B09021_024E']+df['B09021_025E']+
                                              df['B09021_026E'])/df['B09021_022E']+
                                             df['B09020_009E']/df['B09020_001E'])

df['living_with_other65'] = (df['B09021_027E']+df['B09021_028E'])/df['B09021_022E']


# rent/owner costs
df["rent_percent_income65"] = (df["B25072_024E"]*10/df["B25072_023E"] +
                             df["B25072_025E"]*22.5/df["B25072_023E"] +
                             df["B25072_026E"]*27.5/df["B25072_023E"] +
                             df["B25072_027E"]*32.5/df["B25072_023E"] +
                             df["B25072_028E"]*45/df["B25072_023E"])/100
df["owner_cost_percent_income65"] = (df["B25093_024E"]*10/df["B25093_023E"] +
                                     df["B25093_025E"]*22.5/df["B25093_023E"] +
                                     df["B25093_026E"]*27.5/df["B25093_023E"] +
                                     df["B25093_027E"]*32.5/df["B25093_023E"] +
                                     df["B25093_028E"]*45/df["B25093_023E"])/100
df["rent_percent_income"] = df["B25071_001E"]
df["owner_cost_percent_income"] = df["B25092_001E"]

# education
df["education_score"] = ((df["B15003_017E"]+df["B15003_018E"])*2+
                         (df["B15003_019E"]+df["B15003_020E"])*3+
                         df["B15003_021E"]*4+
                         df["B15003_022E"]*5+
                         (df["B15003_023E"]+df["B15003_024E"]+df["B15003_025E"])*6+
                         df["B15003_001E"]-(df["B15003_017E"]+df["B15003_018E"]+df["B15003_019E"]+
                                            df["B15003_020E"]+df["B15003_021E"]+df["B15003_022E"]+
                                            df["B15003_023E"]+df["B15003_024E"]+df["B15003_025E"])
                        )/df["B15003_001E"]

# income/household
df["median_income_above65"] = df["B19049_005E"]
df["median_income"] = df["B19013_001E"]
df["same_address_yrs"] = datetime.datetime.now().year-df["B25039_001E"]

# housing
df['renter_occupied'] = df['B25003_003E']/df['B25003_001E']
df["household_size"] = df["B25010_001E"]
df["house_value"] = df["B25077_001E"]

# insurance

df["insurance_employer_direct"] = (df["B27010_053E"]+df["B27010_054E"]+df["B27010_059E"]+
                                   df["B27010_060E"]+df["B27010_061E"])/df["B27010_051E"]
df["insurance_military_va"] = (df["B27010_056E"]+df["B27010_057E"])/df["B27010_051E"]
df["insurance_none"] = df["B27010_066E"]/df["B27010_051E"]


## Tract level fields

# monthly housing costs
df['monthly_housing_cost'] = df['B25105_001E']

# moved state/county
df['moved_state_county65'] = (df['B07001_062E']+df['B07001_063E']+df['B07001_064E']+
                             df['B07001_078E']+df['B07001_079E']+df['B07001_080E'])/df['B07001_001E']


cols = list(set(df.columns)-set(df_tract.columns))+inc_cols
# display(cols)
df[cols].to_csv(out_path, sep = ',', 
              encoding='utf-8', index=False)
display(df[cols].shape)
display(np.sum(df[cols].isnull())[np.sum(df[cols].isnull())>0].sort_values(ascending=False))
display(np.sum(df[cols]<0)[np.sum(df[cols]<0)>0].sort_values(ascending=False))

(73951, 26)

rent_percent_income65                  5027
median_income_above65                  4077
same_address_yrs                       2287
house_value                            2010
owner_cost_percent_income65            1795
owner_cost_percent_income              1665
rent_percent_income                    1630
moved_state_county65                   1561
living_with_other65                    1144
living_with_spouse_partner_parent65    1144
with_internet                          1144
insurance_military_va                  1103
insurance_none                         1103
insurance_employer_direct              1103
median_income                          1052
monthly_housing_cost                   1015
avg_age_above65                         988
household_size                          902
renter_occupied                         870
education_score                         698
age_62_64                               690
age_above65                             690
dtype: int64

Series([], dtype: int64)

**Transform Blockgroup Level Fields**

In [41]:
df = df_block.copy()
out_path = "output/census_data_blockgroup3.csv"
inc_cols = block_cols

## Blockgroup fields

# % population above 62

df["age_62_64"] = (df['B01001_019E']+df['B01001_043E'])/(df['B01001_002E']+df['B01001_026E'])

df["age_above65"] = (df['B01001_020E']+df['B01001_021E']+df['B01001_022E']+
                     df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
                     df['B01001_044E']+df['B01001_045E']+df['B01001_046E']+
                     df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])/(df['B01001_002E']+df['B01001_026E'])

df["avg_age_above65"] = ((df['B01001_020E']*65.5+df['B01001_021E']*68+df['B01001_022E']*72+
                         df['B01001_023E']*77+df['B01001_024E']*82+df['B01001_025E']*90+
                         df['B01001_044E']*65.5+df['B01001_045E']*68+df['B01001_046E']*72+
                         df['B01001_047E']*77+df['B01001_048E']*82+df['B01001_049E']*90)/
                         (df['B01001_020E']+df['B01001_021E']+df['B01001_022E']+
                         df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
                         df['B01001_044E']+df['B01001_045E']+df['B01001_046E']+
                         df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])
                        )

# presence of computer and type of internet
df["with_internet"] = (df['B28005_016E']+df['B28005_017E'])/df['B28005_014E']

# 65+ living arrangements
df['living_in_group65'] = (df['B09020_021E']>0)
df['living_in_group65'] = df['living_in_group65'].astype(int)
df['living_with_spouse_partner_parent65'] = ((df['B09021_024E']+df['B09021_025E']+
                                              df['B09021_026E'])/df['B09021_022E']+
                                             df['B09020_009E']/df['B09020_001E'])

df['living_with_other65'] = (df['B09021_027E']+df['B09021_028E'])/df['B09021_022E']


# rent/owner costs
df["rent_percent_income65"] = (df["B25072_024E"]*10/df["B25072_023E"] +
                             df["B25072_025E"]*22.5/df["B25072_023E"] +
                             df["B25072_026E"]*27.5/df["B25072_023E"] +
                             df["B25072_027E"]*32.5/df["B25072_023E"] +
                             df["B25072_028E"]*45/df["B25072_023E"])/100
df["owner_cost_percent_income65"] = (df["B25093_024E"]*10/df["B25093_023E"] +
                                     df["B25093_025E"]*22.5/df["B25093_023E"] +
                                     df["B25093_026E"]*27.5/df["B25093_023E"] +
                                     df["B25093_027E"]*32.5/df["B25093_023E"] +
                                     df["B25093_028E"]*45/df["B25093_023E"])/100
df["rent_percent_income"] = df["B25071_001E"]
df["owner_cost_percent_income"] = df["B25092_001E"]

# education
df["education_score"] = ((df["B15003_017E"]+df["B15003_018E"])*2+
                         (df["B15003_019E"]+df["B15003_020E"])*3+
                         df["B15003_021E"]*4+
                         df["B15003_022E"]*5+
                         (df["B15003_023E"]+df["B15003_024E"]+df["B15003_025E"])*6+
                         df["B15003_001E"]-(df["B15003_017E"]+df["B15003_018E"]+df["B15003_019E"]+
                                            df["B15003_020E"]+df["B15003_021E"]+df["B15003_022E"]+
                                            df["B15003_023E"]+df["B15003_024E"]+df["B15003_025E"])
                        )/df["B15003_001E"]

# income/household
df["median_income_above65"] = df["B19049_005E"]
df["median_income"] = df["B19013_001E"]
df["same_address_yrs"] = datetime.datetime.now().year-df["B25039_001E"]

# housing
df['renter_occupied'] = df['B25003_003E']/df['B25003_001E']
df["household_size"] = df["B25010_001E"]
df["house_value"] = df["B25077_001E"]

# insurance

df["insurance_employer_direct"] = (df["B27010_053E"]+df["B27010_054E"]+df["B27010_059E"]+
                                   df["B27010_060E"]+df["B27010_061E"])/df["B27010_051E"]
df["insurance_military_va"] = (df["B27010_056E"]+df["B27010_057E"])/df["B27010_051E"]
df["insurance_none"] = df["B27010_066E"]/df["B27010_051E"]

cols = list(set(df.columns)-set(df_block.columns))+inc_cols
# display(cols)
df[cols].to_csv(out_path, sep = ',', 
              encoding='utf-8', index=False)
display(df[cols].shape)
display(np.sum(df[cols].isnull())[np.sum(df[cols].isnull())>0].sort_values(ascending=False))
display(np.sum(df[cols]<0)[np.sum(df[cols]<0)>0].sort_values(ascending=False))

(220200, 25)

rent_percent_income65                  68603
median_income_above65                  56231
rent_percent_income                    35415
house_value                            13908
owner_cost_percent_income65            10605
owner_cost_percent_income               8768
median_income                           7292
living_with_other65                     3391
living_with_spouse_partner_parent65     3391
with_internet                           3391
insurance_none                          3235
insurance_military_va                   3235
insurance_employer_direct               3235
avg_age_above65                         2963
same_address_yrs                        2887
household_size                          1642
renter_occupied                         1551
education_score                         1145
age_above65                             1106
age_62_64                               1106
dtype: int64

Series([], dtype: int64)

**Blend blockgroup & tract data to reduce nulls**

In [42]:
df_block3 = pd.read_csv('output/census_data_blockgroup3.csv', sep = ',', encoding='utf-8')
df_tract3 = pd.read_csv('output/census_data_tract3.csv', sep = ',', encoding='utf-8')

In [43]:
unique_cols = list(set(df_tract3.columns)-set(df_block3.columns))
c_block = list(set(df_block3.columns)-set(block_cols))

In [44]:
df_block_tract_join = pd.merge(df_block3, df_tract3, how='left',
                               left_on=["state", "county", "tract"], 
                               right_on=["state", "county", "tract"])
display(df_block_tract_join.shape)
display(list(df_block_tract_join.columns))

(220200, 48)

['owner_cost_percent_income65_x',
 'insurance_employer_direct_x',
 'owner_cost_percent_income_x',
 'rent_percent_income_x',
 'renter_occupied_x',
 'age_62_64_x',
 'median_income_x',
 'with_internet_x',
 'household_size_x',
 'education_score_x',
 'insurance_military_va_x',
 'insurance_none_x',
 'living_with_spouse_partner_parent65_x',
 'rent_percent_income65_x',
 'living_with_other65_x',
 'avg_age_above65_x',
 'same_address_yrs_x',
 'living_in_group65_x',
 'house_value_x',
 'age_above65_x',
 'median_income_above65_x',
 'state',
 'county',
 'tract',
 'block group',
 'owner_cost_percent_income65_y',
 'insurance_employer_direct_y',
 'owner_cost_percent_income_y',
 'rent_percent_income_y',
 'renter_occupied_y',
 'age_62_64_y',
 'median_income_y',
 'with_internet_y',
 'household_size_y',
 'education_score_y',
 'insurance_military_va_y',
 'insurance_none_y',
 'living_with_spouse_partner_parent65_y',
 'monthly_housing_cost',
 'living_with_other65_y',
 'rent_percent_income65_y',
 'avg_age_above

In [45]:
df_block_tract = df_block_tract_join.copy()[unique_cols+block_cols]
for c in c_block:
    block_c = c+"_x"
    tract_c = c+"_y"
    df_block_tract[c] = df_block_tract_join[block_c].fillna(df_block_tract_join[tract_c])

display(df_block_tract.shape)
display(np.sum(df_block_tract.isnull())[np.sum(df_block_tract.isnull())>0].sort_values(ascending=False))
display(np.sum(df_block_tract<0)[np.sum(df_block_tract<0)>0].sort_values(ascending=False))

(220200, 27)

rent_percent_income65                  11271
median_income_above65                   6480
moved_state_county65                    3193
house_value                             3017
owner_cost_percent_income65             2789
rent_percent_income                     2461
owner_cost_percent_income               2411
same_address_yrs                        1455
living_with_other65                     1371
living_with_spouse_partner_parent65     1371
with_internet                           1371
insurance_none                          1310
insurance_employer_direct               1310
insurance_military_va                   1310
avg_age_above65                         1176
median_income                           1123
monthly_housing_cost                    1071
household_size                           947
renter_occupied                          913
education_score                          713
age_62_64                                705
age_above65                              705
dtype: int

Series([], dtype: int64)

In [46]:
display(np.sum(df_block3.isnull())[np.sum(df_block3.isnull())>0].sort_values(ascending=False))

rent_percent_income65                  68603
median_income_above65                  56231
rent_percent_income                    35415
house_value                            13908
owner_cost_percent_income65            10605
owner_cost_percent_income               8768
median_income                           7292
living_with_other65                     3391
living_with_spouse_partner_parent65     3391
with_internet                           3391
insurance_none                          3235
insurance_military_va                   3235
insurance_employer_direct               3235
avg_age_above65                         2963
same_address_yrs                        2887
household_size                          1642
renter_occupied                         1551
education_score                         1145
age_above65                             1106
age_62_64                               1106
dtype: int64

In [47]:
df_block_tract.to_csv('output/census_data_blockgroup_tract3.csv', sep = ',',
                      encoding='utf-8', index=False)

**CODE ARCHIVE**

**Calculations**

In [19]:
# # age

# df["age_65_66"] = (df['B01001_020E']+df['B01001_044E'])/(df['B01001_002E']+df['B01001_026E'])
# df["age_67_69"] = (df['B01001_021E']+df['B01001_045E'])/(df['B01001_002E']+df['B01001_026E'])
# df["age_70_74"] = (df['B01001_022E']+df['B01001_046E'])/(df['B01001_002E']+df['B01001_026E'])
# df["age_above75"] = (df['B01001_023E']+df['B01001_024E']+df['B01001_025E']+
#                      df['B01001_047E']+df['B01001_048E']+df['B01001_049E'])/(df['B01001_002E']+df['B01001_026E'])

# # education

# df["education_hs_ged"] = (df["B15003_017E"]+df["B15003_018E"])/df["B15003_001E"]
# df["education_colg"] = (df["B15003_019E"]+df["B15003_020E"])/df["B15003_001E"]
# df["education_assoc_deg"] = df["B15003_021E"]/df["B15003_001E"]
# df["education_bachelors"] = (df["B15003_022E"])/df["B15003_001E"]
# df["education_msandup"] = (df["B15003_023E"]+df["B15003_024E"]+df["B15003_025E"])/df["B15003_001E"]     

# # insurance

# df["insurance_one"] = df["B27010_052E"]/df["B27010_051E"]
# df["insurance_one_employer"] = df["B27010_053E"]/df["B27010_051E"]
# df["insurance_one_direct"] = df["B27010_054E"]/df["B27010_051E"]
# df["insurance_one_medicare"] = df["B27010_055E"]/df["B27010_051E"]
# df["insurance_two"] = df["B27010_058E"]/df["B27010_051E"]
# df["insurance_two_employer_direct"] = df["B27010_059E"]/df["B27010_051E"]
# df["insurance_two_employer_medicare"] = df["B27010_060E"]/df["B27010_051E"]
# df["insurance_two_direct_medicare"] = df["B27010_061E"]/df["B27010_051E"]
# df["insurance_two_medicare_medicaid"] = df["B27010_062E"]/df["B27010_051E"]
# df["insurance_two_private"] = df["B27010_063E"]/df["B27010_051E"]
# df["insurance_two_public"] = df["B27010_064E"]/df["B27010_051E"]
# df["insurance_two_other"] = df["B27010_065E"]/df["B27010_051E"]

# # moved state/county

# df['moved_county_65_69'] = df['B07001_062E']/df['B07001_001E']
# df['moved_county_70_74'] = df['B07001_063E']/df['B07001_001E']
# df['moved_county_above65'] = (df['B07001_062E']+df['B07001_063E']+df['B07001_064E'])/df['B07001_001E']
# df['moved_state_65_69'] = df['B07001_078E']/df['B07001_001E']
# df['moved_state_70_74'] = df['B07001_079E']/df['B07001_001E']
# df['moved_state_above65'] = (df['B07001_078E']+df['B07001_079E']+df['B07001_080E'])/df['B07001_001E']

# # occupation

# df["occupation_management"] = df["B08124_002E"]/df["B08124_001E"]
# df["occupation_service"] = df["B08124_003E"]/df["B08124_001E"]
# df["occupation_sales"] = df["B08124_004E"]/df["B08124_001E"]
# df["occupation_construction"] = df["B08124_005E"]/df["B08124_001E"]
# df["occupation_transportation"] = df["B08124_006E"]/df["B08124_001E"]
# df["occupation_military"] = df["B08124_007E"]/df["B08124_001E"]

# # industry

# df["industry_agriculture"] = df["B08126_002E"]/df["B08126_001E"]
# df["industry_construction"] = df["B08126_003E"]/df["B08126_001E"]
# df["industry_manufacturing"] = df["B08126_004E"]/df["B08126_001E"]
# df["industry_wholesale"] = df["B08126_005E"]/df["B08126_001E"]
# df["industry_retail"] = df["B08126_006E"]/df["B08126_001E"]
# df["industry_transport"] = df["B08126_007E"]/df["B08126_001E"]
# df["industry_information"] = df["B08126_008E"]/df["B08126_001E"]
# df["industry_finance"] = df["B08126_009E"]/df["B08126_001E"]
# df["industry_professional"] = df["B08126_010E"]/df["B08126_001E"]
# df["industry_educational"] = df["B08126_011E"]/df["B08126_001E"]
# df["industry_arts"] = df["B08126_012E"]/df["B08126_001E"]
# df["industry_other"] = df["B08126_013E"]/df["B08126_001E"]
# df["industry_public"] = df["B08126_014E"]/df["B08126_001E"]
# df["industry_military"] = df["B08126_015E"]/df["B08126_001E"]

# # commute

# df["commute_caralone"] = df["B08124_008E"]/df["B08124_001E"]
# df["commute_carpool"] = df["B08124_015E"]/df["B08126_001E"]
# df["commute_public"] = df["B08124_022E"]/df["B08126_001E"]
# df["commute_walked"] = df["B08124_029E"]/df["B08126_001E"]
# df["commute_taxicab"] = df["B08124_036E"]/df["B08126_001E"]
# df["commute_workhome"] = df["B08124_043E"]/df["B08126_001E"]

# # grandparents living with grandkids

# df["grandparents60_working"] = df['B10058_005E']/df['B10058_001E']
# df["grandparents60_not_working"] = df['B10058_010E']/df['B10058_001E']

In [10]:
# ordered_dict = cd.censustable('acs5', 2017, 'B27010')
# reg_dict = dict(ordered_dict)
# fields = list(reg_dict.keys())
# out_dict = {}

# for f in fields:
#     out_dict[f] = reg_dict[f]
    
# pd.DataFrame(out_dict).T.to_csv("extracts/health_insurance_tables.csv", sep = ',',
#                                 encoding='utf-8', index=True)

In [11]:
# c = Census("5c7ae754b8fdea4870fe540ef427a762938fdbe2", year=2017)
# dict_list_block = c.acs5.state_county_blockgroup(fields=('B27001_021E'), state_fips='01', county_fips='001', blockgroup='*')
# pd.DataFrame(dict_list_block)

In [12]:
# c = Census("5c7ae754b8fdea4870fe540ef427a762938fdbe2", year=2017)
# dict_list_tract = c.acs5.state_county_tract(fields=('B27001_021E'),  state_fips='01', county_fips='001', tract='*')
# pd.DataFrame(dict_list_tract)

In [13]:
# # To remove files with no data, 
# # but need a fix in the above loop which will automatically 
# # not write the file is there is no data.

# fnames = glob.glob(path)
# display(len(fnames))

# for i in range(state_county_df.shape[0]):
    
#     state = state_county_df['state_fips'][i]
#     county = state_county_df['county_fips'][i]

#     dict_list_block = c.acs5.state_county_blockgroup(fields=f_block, state_fips=state, 
#                                                      county_fips=county, blockgroup='*')
#     if len(dict_list_block)==0:
#         os.remove("output/data/block_level_%s.csv"%i)
        
#     dict_list_tract = c.acs5.state_county_tract(fields=f_tract, state_fips=state, 
#                                                 county_fips=county, tract='*')
#     if len(dict_list_tract)==0:
#         os.remove("output/data/tract_level_%s.csv"%i)
        
# fnames = glob.glob(path)
# display(len(fnames))

In [14]:
# # To unzip the file uploaded from desktop
# with zipfile.ZipFile("output/census.zip","r") as zip_ref:
#     zip_ref.extractall("output")